In [60]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import rc
from matplotlib.ticker import MultipleLocator, AutoMinorLocator
from lowAltitude_classification.gsd_utils import papermode

In [61]:
font_size = 8
papermode(plt, font_size, has_latex=True)
rc('text.latex', preamble=r'\usepackage{color}')
matplotlib.use('ps')

In [62]:
data = pd.read_csv('../lowAltitude_classification/Scaling_csv/scaling.csv')

num_images = 143055
data['F1'] = data['F1'] * 100

factors = []
fractions = []
for idx, row in data.iterrows():
    fraction = str(row['scale']).replace('.', '/')
    factor = eval(fraction)
    factors.append(factor)
    fractions.append(fraction)
data['scale'] = factors
data['fraction'] = fractions

data['num_images'] = (num_images * data['scale']).astype(int)
data = data.sort_values('num_images')

data

,scale,F1,pAcc,fraction,num_images
4,0.001953,12.22,0.2348,1/512,279
0,0.003906,29.68,0.4547,1/256,558
8,0.007812,32.33,0.4752,1/128,1117
3,0.015625,35.12,0.5012,1/64,2235
5,0.031250,36.07,0.4974,1/32,4470
9,0.062500,38.75,0.5280,1/16,8940
1,0.125000,38.22,0.5247,1/8,17881
10,0.250000,39.13,0.5234,1/4,35763
6,0.500000,39.21,0.5298,1/2,71527
7,0.750000,42.54,0.5403,3/4,107291


In [63]:
width = 3.5
# height = width / 1.618
height = 1.5
fig, ax = plt.subplots(figsize=(width, height))

ax.plot(data['num_images'], data['F1'], color='teal', marker='o')
ax.set_xlabel('Pre-training Set Size (\# images)', labelpad=0)
ax.set_ylabel(r"$F1$ Score on \hspace{2.2em} (\%)")
ax.text(-0.105, 0.75, r"$D_{test}^{drone}$", fontsize=font_size, color="blue", transform=ax.transAxes, ha='center',
        va='center', rotation=90)
ax.set_xscale('log', base=2)
ax.grid(True, which="major", ls="--", color='gray')
ax.set_xlim(200, num_images + 50000)
ax.set_ylim(9, 50)

# xticks bottom
bottom_labels = data['num_images'].tolist()
ax.set_xticks(data['num_images'], bottom_labels, rotation=45, ha='right', rotation_mode="anchor")
ax.tick_params(axis='x', pad=0)
ax.yaxis.set_major_locator(MultipleLocator(10))
ax.yaxis.set_minor_locator(AutoMinorLocator(5))

# xticks top
ax2 = ax.twiny()
ax2.set_xscale('log', base=2)
ax2.set_xlim(ax.get_xlim())
top_labels = []
for i, (n, f) in enumerate(zip(data['num_images'], data['fraction'])):
    top_label = f
    if f == '1/1':
        top_label = '1'
    if f == '3/4':
        top_label = r'$\frac{3}{4}$'
    top_labels.append(top_label)
ax2.set_xticks(data['num_images'], top_labels)

# Lines for other datasets and methods
ax.text(1_200, 37, 'Pseudo-labels', color='chocolate', ha='right', backgroundcolor='white', bbox=dict(facecolor='white', alpha=1, edgecolor='none', boxstyle='round,pad=0.'))
ax.hlines(43.17, 200, 1e6, linestyles='dashed', color='chocolate', label='Pseudo-labels')
ax.text(180_000, 26.0, 'Supervised', color='black', ha='right', backgroundcolor='white', bbox=dict(facecolor='white', alpha=1, edgecolor='none', boxstyle='round,pad=0'))
ax.hlines(32.45, 200, 1e6, linestyles='dashed', color='black', label='Supervised')
ax.text(19_500, 22, r'Soltani \textit{et al.} [10]', color='green', ha='right', backgroundcolor='white', bbox=dict(facecolor='white', alpha=1, edgecolor='none', boxstyle='round,pad=0.'))
ax.vlines(20_523, 9, 50, linestyles='dashed', color='green', label='Soltani et al. [10]')

fig.subplots_adjust(top=0.835, bottom=0.305, left=0.12, right=0.99)

fig.savefig('results/M2F_scaling.pdf')
fig.savefig('results/M2F_scaling.png')

# fig.show()